# **Códificación Shannon Fano**

https://en.wikipedia.org/wiki/Shannon%E2%80%93Fano_coding

*La codificación Shannon-Fano es un algoritmo de compresión de datos sin pérdida desarrollado por Robert Fano a partir de una idea de Claude Shannon.*

**Se trata de una codificación de entropía que produce un código de prefijo muy similar a un código de Huffman , aunque no siempre óptimo, a diferencia de este último.**

Un árbol Shannon-Fano se construye de acuerdo a una especificación diseñada para definir una tabla de códigos efectiva. El algoritmo actual es simple:

1. Para una lista de símbolos dada, crear su correspondiente lista de probabilidades o de frecuencias de aparición de manera que se conozca la frecuencia relativa de ocurrencia de cada símbolo.

2. Ordenar las listas de símbolos de acuerdo a la frecuencia, con los símbolos de ocurrencia más frecuente a la izquierda y los menos comunes a la derecha.

3. Dividir la lista en dos partes, haciendo la frecuencia total de la mitad izquierda lo más próxima posible a la de la mitad derecha.

4. Asignar a la mitad izquierda el dígito binario “0”, y a la mitad derecha el dígito “1”. Esto significa que los códigos para los símbolos en la primera mitad empezarán con “0”, y que los códigos de la segunda mitad empezarán por “1”.
5. Aplicar recursivamente los pasos 3 y 4 a cada una de las dos mitades, subdividiéndolas en grupos y añadiendo bits a los códigos hasta que cada símbolo se corresponde con una hoja del árbol.

### **1. Fuente de información**

In [24]:
# 1. Fuente de información (leyendo un archivo de texto)

def leer_archivo(nombre_archivo):
    try:
        with open(nombre_archivo, encoding="utf8") as archivo:
            contenido = archivo.read()
            return contenido
    except FileNotFoundError:
        print(f"El archivo '{nombre_archivo}' no fue encontrado.")
        return ""

nombre_archivo_fuente = "input.txt"
texto_original = leer_archivo(nombre_archivo_fuente)
print("Mensaje:", texto_original)

Mensaje: No Game No Life Volumen O1

Capı́tulo 1 - Principiante/Amateur

Parte 1
- En un pasado muy, muy lejano.
Los antiguos Deus 1 compitieron por el premio de ser el único Dios supremo y comenzaron
una guerra, junto con sus sirvientes y los seres que creaban.
Esta guerra duró mucho, mucho tiempo, tanto tiempo que se sentía natural.
No había tierra que no estuviera manchada con sangre, y los llantos de agonía nunca
cesaban su eco a través de los cielos.
Las razas inteligentes se odiaban mutuamente, se mataban mutuamente, y seguían
repitiendo incontables asesinatos.
Los elfos tenían pequeñas villas que eran sus bases, y lanzaban magia para cazar a sus
enemigos.
Los dragones utilizaban sus cuerpos para propósitos asesinos, y las bestias de guerra
devoraban a sus presas como animales.
La tierra que era cubierta por el ocaso, se convertía en tierra árida y éstas eran consumidas
por una profunda oscuridad debido a la guerra entre los antiguos Deus.
Los ‘Demon Lords’ 2 y sus monstruos alia

### **2. Transmisor**

### **3. Canal**

### **4. Receptor**

### **5. Destino de información**

In [25]:
# 5. Destino de Información (Destinatario)

mensaje_decodificado = "Hola"

with open("output.txt", "w", encoding="utf-8") as archivo_salida:
    archivo_salida.write(mensaje_decodificado)
print()
print("Mensaje decodificado guardado en 'output'.")



Mensaje decodificado guardado en 'output'.
